In [1]:
from util import *

from src.dataset import load_adult
from src.counterfactual import get_baseline_counterfactuals

import joblib
import time

import warnings
warnings.filterwarnings('ignore')
                        
# Get a model
model, encoder, scaler = joblib.load('models/adult.gz') # Model should have the BlackBox interface
model

2022-11-13 16:18:34.932729: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TabularModel(
  (lin1): Linear(in_features=12, out_features=200, bias=True)
  (lin2): Linear(in_features=200, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=2, bias=True)
  (bn1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drops): Dropout(p=0.3, inplace=False)
)

In [2]:
loader = iter(load_adult(100, train=False))
X_corpus, _ = next(loader)
X_test, _ = next(loader)

simplex = get_simplex(model, X_corpus, X_test, verbose = True)

Weight Fitting Epoch: 2000/10000 ; Error: 6.72 ; Regulator: 39.9 ; Reg Factor: 1
Weight Fitting Epoch: 4000/10000 ; Error: 2.12 ; Regulator: 5.96 ; Reg Factor: 1
Weight Fitting Epoch: 6000/10000 ; Error: 1.36 ; Regulator: 1.63 ; Reg Factor: 1
Weight Fitting Epoch: 8000/10000 ; Error: 1.15 ; Regulator: 0.544 ; Reg Factor: 1
Weight Fitting Epoch: 10000/10000 ; Error: 1.08 ; Regulator: 0.197 ; Reg Factor: 1


In [3]:
%%time
test_id = 10

cfs, x, desired_class = get_simplex_cf_tabular(simplex, model, test_id, encoder)

CPU times: user 477 ms, sys: 6.08 ms, total: 483 ms
Wall time: 60.5 ms


In [4]:
cols = ['workclass', 'education', 'marital-status', 'occupation', \
       'relationship', 'race', 'gender', 'native-country', 'capital-gain', \
       'capital-loss', 'hours-per-week', 'age']

x = simplex.test_examples[test_id:test_id+1]
display_tabular_cfs(cfs, model, x, desired_class, scaler, encoder, cols)

Original: 


,workclass,education,marital-status,occupation,relationship,race,gender,native-country,capital-gain,capital-loss,hours-per-week,age
0,Private,12th,Divorced,Machine-op-inspct,Not-in-family,Black,Female,United-States,0,0,40,41



Kept counterfactual generation: 


,workclass,education,marital-status,occupation,relationship,race,gender,native-country,capital-gain,capital-loss,hours-per-week,age
0,Private,12th,Married-civ-spouse,Machine-op-inspct,Not-in-family,Black,Male,United-States,15024,0,40,54



Predicted:  tensor(1)  ||  Desired:  tensor(1)  ||  Orginal:  tensor(0)
************************************************************************************************************************
Kept counterfactual generation: 


...


Predicted:  tensor(1)  ||  Desired:  tensor(1)  ||  Orginal:  tensor(0)
************************************************************************************************************************
Kept counterfactual generation: 


,workclass,education,marital-status,occupation,relationship,race,gender,native-country,capital-gain,capital-loss,hours-per-week,age
0,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,0,0,45,38



Predicted:  tensor(1)  ||  Desired:  tensor(1)  ||  Orginal:  tensor(0)
************************************************************************************************************************
Kept counterfactual generation: 


,workclass,education,marital-status,occupation,relationship,race,gender,native-country,capital-gain,capital-loss,hours-per-week,age
0,Local-gov,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0,0,40,52



Predicted:  tensor(1)  ||  Desired:  tensor(1)  ||  Orginal:  tensor(0)
************************************************************************************************************************


In [5]:
baseline_cfs = get_baseline_counterfactuals(model = model, target = desired_class, test = x, \
                                            corpus = X_corpus)

display_tabular_cfs(baseline_cfs, model, x, desired_class, scaler, encoder, cols)

Original: 


,workclass,education,marital-status,occupation,relationship,race,gender,native-country,capital-gain,capital-loss,hours-per-week,age
0,Private,12th,Divorced,Machine-op-inspct,Not-in-family,Black,Female,United-States,0,0,40,41



Kept counterfactual generation: 


,workclass,education,marital-status,occupation,relationship,race,gender,native-country,capital-gain,capital-loss,hours-per-week,age
0,Local-gov,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0,0,40,52



Predicted:  tensor(1)  ||  Desired:  tensor(1)  ||  Orginal:  tensor(0)
************************************************************************************************************************


In [6]:
%%time
cf_proto_cf = get_cfproto_cf(X_corpus, model, x)
display_tabular_cfs(cf_proto_cf, model, x, desired_class, scaler, encoder, cols)

2022-11-13 16:18:52.335878: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
No encoder specified. Using k-d trees to represent class prototypes.
2022-11-13 16:18:52.458721: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Original: 


,workclass,education,marital-status,occupation,relationship,race,gender,native-country,capital-gain,capital-loss,hours-per-week,age
0,Private,12th,Divorced,Machine-op-inspct,Not-in-family,Black,Female,United-States,0,0,40,41



Kept counterfactual generation: 


,workclass,education,marital-status,occupation,relationship,race,gender,native-country,capital-gain,capital-loss,hours-per-week,age
0,Private,12th,Married-civ-spouse,Farming-fishing,Not-in-family,Black,Female,United-States,14121,0,43,41



Predicted:  tensor(1)  ||  Desired:  tensor(1)  ||  Orginal:  tensor(0)
************************************************************************************************************************
CPU times: user 4min 27s, sys: 8.83 s, total: 4min 36s
Wall time: 34.7 s


# Comparison

In [7]:
from tqdm import tqdm 

times = []
sparsity = []

for test_id in tqdm(range(20)):
    start = time.time()
    cfs, x, desired_class = get_simplex_cf_tabular(simplex, model, test_id, encoder)
    end = time.time()
    
    start_b = time.time()
    baseline_cfs = get_baseline_counterfactuals(model = model, target = desired_class, test = x, \
                                            corpus = X_corpus)
    end_b = time.time()
    
    start_c = time.time()
    cf_proto_cf = get_cfproto_cf(X_corpus, model, x)
    end_c = time.time()
    
    times.append([end - start, end_b - start_b, end_c - start_c])
    sparsity.append([(cfs[0] != x).sum(), (baseline_cfs[0] != x).sum(), (cf_proto_cf[0] != x).sum()])

100%|███████████████████████████████████████████| 20/20 [12:41<00:00, 38.08s/it]


In [13]:
pd.DataFrame(times, columns  = ['simplex', 'nn', 'cfproto']).describe().to_csv('results/adult_times.csv')

In [19]:
pd.DataFrame(sparsity, columns  = ['simplex', 'nn', 'cfproto']).applymap(int).describe().to_csv('results/adult_sparsity.csv')